# Reflection Pattern

The first pattern we are going to implement is the **reflection pattern**. 

---

<img src="./images/reflection.png" alt="Alt text" width="600"/>

---

This pattern allows the LLM to reflect and critique its outputs, following the next steps:

1. The LLM **generates** a candidate output. If you look at the diagram above, it happens inside the **"Generate"** box.
2. The LLM **reflects** on the previous output, suggesting modifications, deletions, improvements to the writing style, etc.
3. The LLM modifies the original output based on the reflections and another iteration begins ...

**Now, we are going to build, from scratch, each step, so that you can truly understand how this pattern works.**

## Generation Step

### Groq Client and relevant imports

In [1]:
import os
from pprint import pprint
from groq import Groq
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import display_markdown

# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

client = Groq()

In [2]:
def get_llm_client(llm_choice):
    if llm_choice == "GROQ":
        client = OpenAI(
            base_url="https://api.groq.com/openai/v1",
            api_key=os.environ.get("GROQ_API_KEY"),
        )
        return client
    elif llm_choice == "OPENAI":
        load_dotenv()  # load environment variables from .env fil
        client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
        return client
    else:
        raise ValueError("Invalid LLM choice. Please choose 'GROQ' or 'OPENAI'.")

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

# LLM_CHOICE = "OPENAI"
LLM_CHOICE = "GROQ"

if OPENAI_API_KEY:
    print(f"OPENAI_API_KEY exists and begins {OPENAI_API_KEY[:14]}...")
else:
    print("OPENAI_API_KEY not set")

if GROQ_API_KEY:
    print(f"GROQ_API_KEY exists and begins {GROQ_API_KEY[:14]}...")
else:
    print("GROQ_API_KEY not set")


client = get_llm_client(LLM_CHOICE)
if LLM_CHOICE == "GROQ":
    MODEL = "llama-3.3-70b-versatile"
else:
    MODEL = "gpt-4o-mini"

print(f"LLM_CHOICE: {LLM_CHOICE} - MODEL: {MODEL}")

OPENAI_API_KEY exists and begins sk-proj-1WUVgv...
GROQ_API_KEY exists and begins gsk_11hFN1EMfj...
LLM_CHOICE: GROQ - MODEL: llama-3.3-70b-versatile


We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python 
programmer eager to receive feedback / critique by the user.

In [5]:
generation_chat_history = [
    {
        "role": "system",
        "content": "You are a Python programmer tasked with generating high quality Python code."
        "Your task is to Generate the best content possible for the user's request. If the user provides critique," 
        "respond with a revised version of your previous attempt."
    }
]

Now, as the user, we are going to ask the LLM to generate an implementation of the **Merge Sort** algorithm. Just add a new message with the **user** role to the chat history.

In [6]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Create an AI Agent that does reflection."
    }
)

Let's generate the first version of the essay.

In [7]:
generated_code = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

generation_chat_history.append(
    {
        "role": "assistant",
        "content": generated_code
    }
)

In [8]:
display_markdown(generated_code, raw=True)

Here's an example of a basic AI agent that performs reflection in Python:
```
import inspect
import types

class ReflectionAI:
    def __init__(self):
        self.knowledge_base = {}

    def learn(self, func):
        """
        Learn a new function and store its metadata in the knowledge base.
        """
        self.knowledge_base[func.__name__] = {
            'docstring': func.__doc__,
            'args': inspect.getfullargspec(func).args,
            'source_code': inspect.getsource(func)
        }

    def reflect(self, func_name):
        """
        Reflect on a learned function and return its metadata.
        """
        if func_name in self.knowledge_base:
            return self.knowledge_base[func_name]
        else:
            return None

    def introspect(self):
        """
        Return a list of all learned functions and their metadata.
        """
        return self.knowledge_base

    def think(self):
        """
        Perform some thinking/reflection on the learned functions.
        """
        # For now, just print out the introspection results
        print("Thinking...")
        print(self.introspect())

def add(a, b):
    """Adds two numbers."""
    return a + b

def greeting(name):
    """Says hello to someone."""
    print(f"Hello, {name}!")

ai = ReflectionAI()
ai.learn(add)
ai.learn(greeting)

print(ai.reflect('add'))
# Output: {'docstring': 'Adds two numbers.', 'args': ['a', 'b'], 'source_code': 'def add(a, b):\n    """Adds two numbers."""\n    return a + b\n'}

ai.think()
# Output: {'add': {'docstring': 'Adds two numbers.', 'args': ['a', 'b'], 'source_code': 'def add(a, b):\n    """Adds two numbers."""\n    return a + b\n'}, 'greeting': {'docstring': 'Says hello to someone.', 'args': ['name'], 'source_code': 'def greeting(name):\n    """Says hello to someone."""\n    print(f"Hello, {name}!")\n'}}
```
This AI agent has four main methods:

1. `learn(func)`: Learns a new function and stores its metadata (docstring, arguments, and source code) in the knowledge base.
2. `reflect(func_name)`: Reflects on a learned function and returns its metadata.
3. `introspect()`: Returns a list of all learned functions and their metadata.
4. `think()`: Performs some thinking/reflection on the learned functions (currently just prints out the introspection results).

The agent learns two sample functions, `add` and `greeting`, and then performs reflection and introspection on them.

Please let me know if this meets your expectations or if you'd like me to revise anything!

## Reflection Step

Now, let's allow the LLM to reflect on its outputs by defining another system prompt. 

In [9]:
reflection_chat_history = [
    {
    "role": "system",
    "content": "You are an experienced Python code reviwer. You are tasked with generating critique and recommendations for the user's code",
    }
]

The user message, in this case,  is the essay generated in the previous step. We simply add the `generated_codee` to the `reflection_chat_history`.

In [10]:
reflection_chat_history.append(
    {
        "role": "user",
        "content": generated_code
    }
)

Now, let's generate a critique to the Python code.

In [14]:
critique = client.chat.completions.create(
    model=MODEL, messages=generation_chat_history, temperature=0.7
).choices[0].message.content

In [15]:
display_markdown(critique, raw=True)

 

Note: The above code uses Python's built-in `inspect` module to extract metadata from the learned functions. The `inspect` module provides several useful functions for inspecting live objects such as modules, classes, objects, etc. 

### Advanced Usage

To make this agent more advanced, you could add the following features:

*   **Learning from other sources**: Currently, the agent only learns from functions passed directly to it. You could modify it to learn from other sources, such as files, databases, or even other AI agents.
*   **More advanced thinking/reflection**: The `think()` method currently just prints out the introspection results. You could modify it to perform more complex thinking/reflection tasks, such as analyzing the relationships between learned functions or generating new functions based on the learned knowledge.
*   **Error handling and validation**: The agent currently assumes that all learned functions are valid and can be introspected successfully. You could add error handling and validation to handle cases where this is not true.
*   **Improved knowledge representation**: The agent currently stores learned functions in a simple dictionary. You could modify it to use a more advanced knowledge representation, such as a graph or ontology, to store and reason about the learned knowledge.

### Example Use Cases

Here are some example use cases for this AI agent:

*   **Automated programming**: The agent could be used to automate the process of learning and reflecting on code. For example, it could be used to generate documentation for a large codebase or to identify patterns and relationships between different parts of the code.
*   **Code analysis and optimization**: The agent could be used to analyze and optimize code. For example, it could be used to identify performance bottlenecks or to suggest improvements to the code.
*   **Education and training**: The agent could be used to educate and train developers. For example, it could be used to generate interactive coding exercises or to provide personalized feedback and guidance to developers.

Please let me know if you would like me to revise or add anything to the above code. I am here to help.

Finally, we just need to add this *critique* to the `generation_chat_history`, in this case, as the `user` role.

In [10]:
generation_chat_history.append(
    {
        "role": "user",
        "content": critique
    }
)

## Generation Step (II)

In [16]:
essay = client.chat.completions.create(
    model=MODEL, messages=generation_chat_history, temperature=0.7
).choices[0].message.content

In [17]:
display_markdown(essay, raw=True)

 

### How does this code work?
1. **Initialization**: An empty `knowledge_base` dictionary is created to store metadata about learned functions.
2. **Learning**: The `learn` function takes another function as input and extracts its metadata using the `inspect` module. This metadata is then stored in the `knowledge_base`.
3. **Reflection**: The `reflect` function takes a function name as input and returns the corresponding metadata from the `knowledge_base`.
4. **Introspection**: The `introspect` function returns the entire `knowledge_base`, which contains metadata for all learned functions.
5. **Thinking**: The `think` function currently just prints out the results of introspection, but this is where you could implement more complex thinking/reflection logic.

### Possible improvements
* **Add more thinking/reflection logic**: Currently, the `think` function just prints out the introspection results. You could add more complex logic here to analyze the learned functions and their metadata.
* **Support for more advanced function metadata**: The current implementation only extracts basic metadata like docstrings and argument names. You could add support for more advanced metadata, such as function return types or docstring parsing.
* **Error handling**: The current implementation assumes that the input functions are valid and can be learned. You could add error handling to handle cases where the input functions are invalid or cannot be learned.
* **Support for other types of knowledge**: The current implementation only supports learning functions. You could add support for learning other types of knowledge, such as data structures or classes.

## And the iteration starts again ...